In [ ]:
#EXP4
!pip install SpeechRecognition
!pip install googletrans==4.0.0-rc1
import speech_recognition as sr
from googletrans import Translator
import numpy as np
import tensorflow as tf
from tensorflow import keras

recognizer = sr.Recognizer()

audio_file_path = "/content/quothello-therequot-158832.wav"

target_language = "en"

translator = Translator()
with sr.AudioFile(audio_file_path) as source:
        print("Listening to the audio...")
        audio_data = recognizer.record(source)
        print("Audio captured. Transcribing...")
        recognized_text = recognizer.recognize_google(audio_data)
        print("Recognized Text: ", recognized_text)
        translation = translator.translate(recognized_text, src="auto", dest=target_language)
        translated_text = translation.text

        print("Translated Text: ", translated_text)
chars = sorted(list(set(translated_text)))
char_to_index = {char: index for index, char in enumerate(chars)}
index_to_char = {index: char for index, char in enumerate(chars)}
n_gram = 3
batch_size = 64
epochs = 100
input_sequences = []
output_sequences = []

for i in range(0, len(translated_text) - n_gram, 1):
    input_seq = translated_text[i:i + n_gram]
    output_seq = translated_text[i + n_gram]
    input_sequences.append(input_seq)
    output_sequences.append(output_seq)
if len(input_sequences) == 0:
    raise ValueError("Training dataset is empty.")

X = np.zeros((len(input_sequences), n_gram, len(chars)), dtype=np.bool_)
y = np.zeros((len(input_sequences), len(chars)), dtype=np.bool_)

for i, sequence in enumerate(input_sequences):
    for t, char in enumerate(sequence):
        X[i, t, char_to_index[char]] = 1
    y[i, char_to_index[output_sequences[i]]] = 1
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(128, input_shape=(n_gram, len(chars))))
model.add(keras.layers.Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y, batch_size=batch_size, epochs=epochs)
start_index = np.random.randint(0, len(translated_text) - n_gram)
seed_text = translated_text[start_index:start_index + n_gram]
generated_text = seed_text

for i in range(400):  # Generate 400 characters
    sampled = np.zeros((1, n_gram, len(chars)))
    for t, char in enumerate(seed_text):
        sampled[0, t, char_to_index[char]] = 1.

    predictions = model.predict(sampled, verbose=0)[0]
    next_index = np.argmax(predictions)
    next_char = index_to_char[next_index]

    generated_text += next_char
    seed_text = seed_text[1:] + next_char
print(generated_text)
test_loss = model.evaluate(X, y, verbose=0)

accuracy = 1.0 - test_loss
print(f"Test Accuracy: {accuracy * 100:.2f}%")
